In [1]:
import pandas as pd
import numpy as np
import os, random, csv
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import scipy.sparse as ss
from htmlparsing_kit import clean_sections

def clean_members(x,mdict):
    judges = []
    for item in x.split("_"):
        item = item.strip()
        if item in list(mdict.keys()):
            judges.append(str(mdict[item]))
        else:
            continue
    judges = '_'.join(list(set(judges)))
    return judges 

def clean_sections2(x):
    x = str(x)
    if len(x) <100:
        newc = x
    else:
        newc=''
    if newc == 'nan':
        newc = ''
    else:
        newc = x
    return newc

def map_vectwords(x,words):
    pwords = []
    for w in words:
        if w in x:
            pwords.append(w)
        else:
            continue
    pwords = sorted(list(set(pwords)))
    return ', '.join(pwords)

def keywordstotext(row):
    kws = row['keywords'].split(',')
    t = row['topiccleantext']
    torderdict = {}
    for kw in kws:
        kw = str(kw).strip()
        if t.find(kw) != -1:
            torderdict[t.find(kw)] = kw
        else:
            continue
    torderdict = dict(sorted(torderdict.items()))
    kwordered = ' '.join(list(torderdict.values()))
    return kwordered

In [2]:
mainpath = str(os.getcwd())+'/'

df = pd.read_csv(mainpath+"output/5_model_results.csv",sep='\t')
df['cleansections']=df['sections'].apply(lambda x: clean_sections(x))
df['datetime'] = pd.to_datetime(df['date'])
df['year'] =df['datetime'].dt.year
df['quarter'] = pd.PeriodIndex(df.datetime, freq='Q')
df['monthyear'] = pd.PeriodIndex(df.datetime, freq='M')
df['numone'] = 1
print(df.shape)
df.fillna(np.nan,inplace=True)
df = df.sample(frac=1).reset_index(drop=True)

df['id'] = list(range(0,df.shape[0]))

members = [x for x in list(set([x.strip() for x in '_'.join(list(set(df['member'].tolist()))).split('_')])) if 'ottawa' not in x.lower()]
random.shuffle(members)
mdict = dict(zip(members,list(range(0,len(members)))))
print(len(mdict))
df['cleanmember'] = df['member'].apply(lambda x:clean_members(x,mdict))
df['csections'] = df['cleansections'].apply(lambda x: clean_sections2(x))

docs = df['topiccleantext'].tolist()
vectorizern = TfidfVectorizer(ngram_range=(1, 1), max_features = 6000, max_df = 0.95, min_df = 0.05)
dtm_tf = vectorizern.fit_transform(docs)
dtm_tf = ss.csr_matrix(dtm_tf)
words = vectorizern.get_feature_names_out()

df['keywords'] = df['topiccleantext'].apply(lambda x: map_vectwords(x,words))
df['dbtext'] = df.apply(lambda x: keywordstotext(x),axis=1)

(36461, 35)
163


In [4]:
pdf = df[['year','id','date','newloc','cleanmember','csections','applicant','winner','dbtext']]
pdf.columns = ['year','Case Id','Date','Office Location','Adjudicators','Act Sections','Applicant','Winner','Order Text']

In [1]:
for n,g in pdf.groupby('year'):
    publicdbpath = mainpath + 'output/publicdb/onltbdb_'+str(n)+'.csv'
    # print("csvtotable "+ publicdbpath.split('/')[-1] + " --caption " + '"Ontario LTB Orders ('+str(n)+')" '+ publicdbpath.split('/')[-1].split('.')[0].strip()+'.html')
    g = g[['Case Id','Date','Office Location','Adjudicators','Act Sections','Applicant','Winner','Order Text']]
    g.to_csv(publicdbpath,sep=',',index=False,quoting=csv.QUOTE_ALL)
# example for converting csvs to searchable html tables:
# csvtotable onltbdb_2006.csv --caption "Ontario LTB Orders (2006)" onltbdb_2006.html

In [7]:
pdf.to_csv(mainpath + 'output/onltbdb_2006_2021.csv',sep=',',index=False)